In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

# Any results you write to the current directory are saved as output.

In [2]:
#!pip install '../input/dlibpkg/dlib-19.19.0'

In [3]:
#!pip install '../input/face-recognition/click-7.1.1-py2.py3-none-any.whl'
#!pip install '/kaggle/input/face-recognition/Pillow-7.0.0-cp38-cp38-win32.whl'
#!pip install '../input/needwheels/face_recognition_models-0.3.0-py2.py3-none-any.whl'
#!pip install '../input/needwheels/face_recognition-1.3.0-py2.py3-none-any.whl'

In [1]:
import dlib
import cv2
import os
import re
import json
import numpy as np
import pandas as pd
from PIL import Image, ImageChops, ImageEnhance, ImageDraw
import face_recognition


import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [5]:
try:
    os.makedirs('../working/dataset/TEST')
except:
    pass

In [6]:
test = '../input/deepfake-detection-challenge/test_videos/'
test_video_files = [test + x for x in sorted(os.listdir(test))]
test_video_files = sorted(test_video_files)
test_image_path = '../working/dataset/TEST/'
test_image_files = [test_image_path + x for x in sorted(os.listdir(test_image_path))]

test_videos = '../input/deepfake-detection-challenge/test_videos/'
test_movie_files = [test_videos + x for x in sorted(os.listdir(test_videos))]

dic = {i.split('/')[-1]:[0,0] for i in test_movie_files}

In [2]:
model = load_model('../input/kerasmodel/deepfake-detection-model.h5')
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 2, 2, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 

In [14]:
detector = dlib.get_frontal_face_detector()
vid_num = 0
all_vid = len(test_video_files)
for vid in test_video_files:
    count = 0
    frame = 0
    file_name_mp4 = vid.split('/')[-1]
    file_name = file_name_mp4.split('.')[0]
    cap = cv2.VideoCapture(vid)
    while(cap.isOpened()):
        ret = cap.grab()
        if ret == False:
            break
        if frame % 30 == 0:
            ret, image = cap.retrieve()
            face_rects, scores, idx = detector.run(image, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                li = [x1,y1,x2,y2]
                li = [0 if i<0 else li[idx] for idx, i in enumerate(li)]
                x1,y1,x2,y2 = li
                
                crop_img = image[y1:y2, x1:x2]
                landmarks = face_recognition.face_landmarks(crop_img)
                pil = Image.fromarray(crop_img)
                p = ImageDraw.Draw(pil)
                for idx, landmark in enumerate(landmarks):
                    for mark in landmark.keys():
                        p.line(landmark[mark], width=3)
                        
                
                
                pix = np.array(cv2.resize(np.array(pil), (128, 128)))
                pix = pix[np.newaxis] # 차원 확장
                pred = model.predict(pix)[0]
                pred[1]
                filename = file_name+'.mp4'
                dic[filename][0] += pred
                dic[filename][1] += 1
                
                
                # cv2.imwrite(f'../working/dataset/TEST/{file_name}_{count}.jpg', cv2.resize(np.array(pil), (128, 128)))
                count += 1
        frame += 1
        #test
        #break
        
        
    if vid_num%5 == 4:
        print(dic[filename])
        print(f'{vid_num}/{all_vid}')
    vid_num += 1
    cap.release()

KeyboardInterrupt: 

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict(dic, orient='index',columns=['real', 'num_img'])
df.index.name='filename'
df['label'] = (df['real']/df['num_img']).fillna(0.5) # 평균내기 # 얼굴 인식 못한 사진은 0.5로
df

In [ ]:
df2=df[['label']]
df2

In [ ]:
df2.loc[df2['label']==0] = 0.01
df2.loc[df2['label']==1] = 0.09
df2

In [ ]:
df2.to_csv('submission.csv') # csv 저장